In [37]:
#import required libraries
import pandas as pd


In [38]:
#get the sentiment dataset
df_sentiment_analysis = pd.read_csv(r'C:\Users\user\Desktop\Vinni study folder\my python notebooks\practice datasets\imdb_labelled.txt',sep='\t',names=['comment','label'])

In [39]:
#view first 10 observations. 
# 1 indicates positive sentiment and 0 indicate negative sentiment
df_sentiment_analysis.head()

,comment,label
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [40]:
# view more information about the setiment data using describe method
df_sentiment_analysis.describe()

,label
count,748.000000
mean,0.516043
std,0.500077
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [41]:
#view more info on data
df_sentiment_analysis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  748 non-null    object
 1   label    748 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 11.8+ KB


In [42]:
# view data using group by and describe method
df_sentiment_analysis['label'].describe()

count    748.000000
mean       0.516043
std        0.500077
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: label, dtype: float64

In [43]:
# Verify length of the messages and also add it also as a new column (feature)
df_sentiment_analysis['length'] = df_sentiment_analysis['comment'].apply(len)

In [44]:
# view first 5 messages with length
df_sentiment_analysis.head()

,comment,label,length
0,"A very, very, very slow-moving, aimless movie ...",0,87
1,Not sure who was more lost - the flat characte...,0,99
2,Attempting artiness with black & white and cle...,0,188
3,Very little music or anything to speak of.,0,44
4,The best scene in the movie was when Gerardo i...,1,108


In [45]:
#view first 
df_sentiment_analysis = df_sentiment_analysis.sort_values('length',ascending = True)

In [46]:
# start text processing with vectorizer 
df_sentiment_analysis

,comment,label,length
64,10/10,1,7
590,10/10,1,7
101,Awful.,0,8
295,See it.,1,9
655,Hated it.,0,11
...,...,...,...
135,You'll love it! \t1\nThis movie is BAD. \t0\...,1,1053
149,"I love it. \t1\nThe ending is so, SO perfect...",0,1562
646,"With great sound effects, and impressive spec...",1,4488
19,The structure of this film is easily the most...,0,4779


In [49]:
import string
from nltk.corpus import stopwords

# define a function to get rid of stopwords present in the messages
def remove_stopwords(mess):
    no_punctuation = [char for char in mess if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return [char for char in no_punctuation.split() if char.lower() not in stopwords.words('english') ]

In [53]:
from sklearn.feature_extraction.text import CountVectorizer

# bag of words by applying the function and fit the data (comment) into it
bag_of_words = CountVectorizer(remove_stopwords).fit(df_sentiment_analysis['comment'])

In [54]:
# apply transform method for the bag of words
comment_bagofwords = bag_of_words.transform(df_sentiment_analysis['comment'])

In [55]:
# apply tfidf transformer and fit the bag of words into it (transformed version)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(comment_bagofwords)

In [57]:
# print shape of the tfidf 
comment_tfidf = tfidf_transformer.transform(comment_bagofwords)
print (comment_tfidf.shape)

(748, 3051)


In [59]:
#choose naive Bayes model to detect the spam and fit the tfidf data into it
from sklearn.naive_bayes import MultinomialNB
sentiment_detection_model = MultinomialNB().fit(comment_tfidf,df_sentiment_analysis['label'])

In [64]:
# check model for the predicted  and expected value say for comment# 1 and comment#5
comment = df_sentiment_analysis['comment'][10]
bag_of_words_for_comment = bag_of_words.transform([comment])
tfidf = tfidf_transformer.transform(bag_of_words_for_comment)

print ('predicted sentiment label ', sentiment_detection_model.predict(tfidf)[0])
print ('expected sentiment label', df_sentiment_analysis.label[10])

predicted sentiment label  1
expected sentiment label 1
